# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [74]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [75]:
# Create a DataFrame

VacationPy = pd.read_csv("../WeatherPy Challenge/Resources/weatherdata.csv")

VacationPy.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Fortaleza,-3.7227,-38.5247,80.73,78,75,13.80,BR,1658702145
1,1,Kavaratti,10.5669,72.6420,81.16,75,100,14.29,IN,1658702396
2,2,Airai,-8.9266,125.4092,57.13,55,56,2.46,TL,1658702241
3,3,East London,-33.0153,27.9116,59.13,62,74,7.67,ZA,1658702244
4,4,Saint-Philippe,-21.3585,55.7679,65.21,80,44,9.22,RE,1658702242


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [76]:
# Remove rows with missing values

VacationPy = VacationPy.dropna()
VacationPy.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Fortaleza,-3.7227,-38.5247,80.73,78,75,13.80,BR,1658702145
1,1,Kavaratti,10.5669,72.6420,81.16,75,100,14.29,IN,1658702396
2,2,Airai,-8.9266,125.4092,57.13,55,56,2.46,TL,1658702241
3,3,East London,-33.0153,27.9116,59.13,62,74,7.67,ZA,1658702244
4,4,Saint-Philippe,-21.3585,55.7679,65.21,80,44,9.22,RE,1658702242


In [77]:
# Store 'Lat' and 'Lng' into  locations 
locations = VacationPy[["Lat", "Lng"]].astype(float)

In [78]:
# Convert humidity to float and store
# HINT: be sure to handle NaN values
VacationPy = VacationPy.dropna()
Humidity = VacationPy["Humidity"].astype(float)

In [79]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [80]:
# Cities with good weather conditions

# max temp : >65 and <75
# humidity : < 60
# Cloudiness : < 45


#drop the rows that do not satisfy the conditions above

VacationPy.drop(VacationPy[(VacationPy["Max Temp"] < 65) | (VacationPy["Max Temp"] > 75)].index, inplace = True)
VacationPy.drop(VacationPy[VacationPy["Humidity"] > 60].index, inplace = True)
VacationPy.drop(VacationPy[VacationPy["Cloudiness"] > 45].index, inplace = True)
VacationPy                           

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
159,159,Yumen,40.2833,97.2000,73.35,34,24,8.50,CN,1658702316
195,195,Churapcha,62.0025,132.4325,72.95,55,0,4.38,RU,1658702429
214,214,Caçu,-18.5567,-51.1308,74.03,35,9,5.70,BR,1658702432
231,231,Zhezkazgan,47.8043,67.7144,72.14,30,28,11.25,KZ,1658702436
235,235,Ust-Nera,64.5667,143.2000,67.32,50,2,2.80,RU,1658702392
269,269,Kaputa,-8.4689,29.6619,68.74,52,9,1.63,ZM,1658702444
273,273,Zlobin,45.2924,14.6541,70.38,60,0,6.93,HR,1658702445
281,281,Avarua,-21.2078,-159.7750,73.45,53,20,5.75,CK,1658702238
308,308,Harsīn,34.2721,47.5861,70.61,21,0,0.89,IR,1658702452
316,316,Arrondissement de Carpentras,44.0833,5.1667,73.92,56,0,3.09,FR,1658702351


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [81]:
hotel_df = pd.DataFrame(VacationPy)
hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
159,159,Yumen,40.2833,97.2000,73.35,34,24,8.50,CN,1658702316,
195,195,Churapcha,62.0025,132.4325,72.95,55,0,4.38,RU,1658702429,
214,214,Caçu,-18.5567,-51.1308,74.03,35,9,5.70,BR,1658702432,
231,231,Zhezkazgan,47.8043,67.7144,72.14,30,28,11.25,KZ,1658702436,
235,235,Ust-Nera,64.5667,143.2000,67.32,50,2,2.80,RU,1658702392,
269,269,Kaputa,-8.4689,29.6619,68.74,52,9,1.63,ZM,1658702444,
273,273,Zlobin,45.2924,14.6541,70.38,60,0,6.93,HR,1658702445,
281,281,Avarua,-21.2078,-159.7750,73.45,53,20,5.75,CK,1658702238,
308,308,Harsīn,34.2721,47.5861,70.61,21,0,0.89,IR,1658702452,
316,316,Arrondissement de Carpentras,44.0833,5.1667,73.92,56,0,3.09,FR,1658702351,


In [83]:
#

baseURL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#search for the hotels

target_radius = 5000
target_type = "hotel"


# dictionary of parameters to update at each iteration 
parameters = {
    "type": target_type,
    "radius" :target_radius,
    "key": g_key
}

# use iterrows() to iterate through the rows in the DF
for index, row in hotel_df.iterrows():
    #grab the latitudes and longitudes from the columns in the DF
    latitude = row['Lat']
    longitude = row['Lng']
    
    # change location each iteration while leaving original params in place
    parameters["location"] = f"{latitude},{longitude}"
    
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_response = requests.get(base_url, params=parameters)
  

    # convert to json
    hotel_json = hotel_response.json()
    #pprint(hotel_json) # to check the indices of the hotel name 
    
 
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_json["results"][1]["name"]
        print(f"First hotel in {target_radius} meters is {hotel_json['results'][1]['name']}")
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    print("--------------------------------------")


First hotel in 5000 meters is Xiadong No. Liudui
--------------------------------------
First hotel in 5000 meters is Chuoranchyk
--------------------------------------
First hotel in 5000 meters is City of Caçu
--------------------------------------
First hotel in 5000 meters is Hotel Zumrat
--------------------------------------
First hotel in 5000 meters is Sarylakh-Sur'ma
--------------------------------------
First hotel in 5000 meters is Kaputa Town Council
--------------------------------------
First hotel in 5000 meters is Hotel Bitoraj Fuzine
--------------------------------------
First hotel in 5000 meters is Paradise Inn
--------------------------------------
First hotel in 5000 meters is Mehr Eqtesad Bank
--------------------------------------
First hotel in 5000 meters is Moulin d'Antelon
--------------------------------------
First hotel in 5000 meters is Kingdom Hall of Jehova's Witnesses
--------------------------------------
First hotel in 5000 meters is Australia Post

In [84]:
# checking the DF with the hotel names in the new column
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
159,159,Yumen,40.2833,97.2000,73.35,34,24,8.50,CN,1658702316,Xiadong No. Liudui
195,195,Churapcha,62.0025,132.4325,72.95,55,0,4.38,RU,1658702429,Chuoranchyk
214,214,Caçu,-18.5567,-51.1308,74.03,35,9,5.70,BR,1658702432,City of Caçu
231,231,Zhezkazgan,47.8043,67.7144,72.14,30,28,11.25,KZ,1658702436,Hotel Zumrat
235,235,Ust-Nera,64.5667,143.2000,67.32,50,2,2.80,RU,1658702392,Sarylakh-Sur'ma


In [85]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [86]:
# adding markers to the heatmap created 
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)


# Add layer
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))